In [1]:
import os
import sys
sys.path.append("/home/dux/")
sys.path.append("/home/dux/surface_sampling/sgmc_surf")

from lammps import lammps
from ase.calculators.lammpsrun import LAMMPS, write_lammps_in
from ase.calculators.lammps import Prism
from ase import io

from mcmc import mcmc_run, slab_energy

In [2]:
# try 2003 tersoff potential 
parameters = {
    'pair_style': 'tersoff',
    'pair_coeff': ['* * GaN.tersoff Ga N']
}
potential_file = os.path.join(os.environ["LAMMPS_POTENTIALS"], 'GaN.tersoff')
lammps_calc = LAMMPS(files=[potential_file], keep_tmp_files=False, keep_alive=False, tmp_dir="/home/dux/surface_sampling/tmp_files")
lammps_calc.set(**parameters)

slab = io.read("/home/dux/surface_sampling/sgmc_surf/GaN/GaN_0001_3x3_12_Ga_ads_arranged_slab.cif")

In [3]:
# write lammps data file
lammps_data_file = "/home/dux/surface_sampling/sgmc_surf/py_lammps/lammps.data"
lammps_in_file = "/home/dux/surface_sampling/sgmc_surf/py_lammps/lammps.in"
potential_file = "/home/dux/surface_sampling/sgmc_surf/py_lammps/GaN.tersoff"
atoms = ["Ga", "N"]
lammps_out_file = "/home/dux/surface_sampling/sgmc_surf/py_lammps/lammps.out"
cif_from_lammps_path = "/home/dux/surface_sampling/sgmc_surf/py_lammps/lammps.cif"

# write current surface into lammps.data
slab.write(lammps_data_file, format="lammps-data", units="real", atom_style="atomic")

In [4]:
TEMPLATE = """
clear
atom_style atomic 
units metal
boundary p p p 
atom_modify sort 0 0.0 

# read_data /path/to/data.data
read_data {}

### set bulk
group bulk id <= 36

### interactions
pair_style tersoff 
# pair_coeff * * /path/to/potential Atom1 Atom2
pair_coeff * * {} {} {} 
mass 1 69.723000 
mass 2 14.007000 

### run
reset_timestep 0
fix 2 bulk setforce 0.0 0.0 0.0
thermo 10 # output thermodynamic variables every N timesteps

thermo_style custom step temp press cpu pxx pyy pzz pxy pxz pyz ke pe etotal vol lx ly lz atoms
thermo_modify flush yes format float %23.16g
min_style cg 
minimize 1e-25 1e-25 5000 10000 

# write_data /path/to/data.out
write_data {}
print "_end of energy minimization_" 
log /dev/stdout

"""

# write lammps.in file
with open(lammps_in_file, 'w') as f:
    f.writelines(TEMPLATE.format(lammps_data_file, potential_file, *atoms, lammps_out_file))

In [5]:
# cell = slab.get_cell()
# prism = Prism(cell)

# write_lammps_in(
#             lammps_in=lammps_in_file,
#             parameters=lammps_calc.parameters,
#             atoms=slab,
#             prismobj=prism
#         )

# # TODO: give up writing LAMMPSIN using these things

# write_lammps_in(lammps_in=lammps_in_file, lammps_data=lammps_data_file)

In [6]:
lmp = lammps()
# print("LAMMPS Version: ", lmp.version())

# run the LAMMPS here
lmp.file(lammps_in_file)
lmp.close()

# Read from LAMMPS out
opt_slab = io.read(lammps_out_file, format='lammps-data', style='atomic')

atomic_numbers_dict = {1: 31, 2: 7} # 1: Ga, 2: N
actual_atomic_numbers = [atomic_numbers_dict[x] for x in opt_slab.get_atomic_numbers()]
print(f"actual atomic numbers {actual_atomic_numbers}")
# correct_lammps = new_slab.copy()
opt_slab.set_atomic_numbers(actual_atomic_numbers)

LAMMPS (29 Sep 2021)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
LAMMPS Version:  20210929
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (0.0000000 0.0000000 0.0000000) to (9.6488700 8.3561665 23.265800) with tilt (-4.8244350 0.0000000 0.0000000)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  48 atoms
  read_data CPU = 0.001 seconds
36 atoms in group bulk
Reading tersoff potential file /home/dux/surface_sampling/sgmc_surf/py_lammps/GaN.tersoff with DATE: 2007-10-22
Neighbor list info ...
  update every 1 steps, delay 0 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 5.1
  ghost atom cutoff = 5.1
  binsize = 2.55, bins = 6 4 10
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair tersoff, perpetual
      attributes: full, newton 

In [7]:
# set new positions for original slab
slab.set_positions(opt_slab.get_positions())
# slab.write(cif_from_lammps_path)

In [ ]:
nlocal = lmp.extract_global("nlocal")
x = lmp.extract_atom("x")

for i in range(nlocal):
   print("(x,y,z) = (", x[i][0], x[i][1], x[i][2], ")")